<a href="https://colab.research.google.com/github/arkincognito/PyTorch/blob/main/03_Pytorch_Multivariable_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# Define training dataset
>```
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
```



# Weights and Biases

Hypothesis:
$$ H(x)= x\times W + b = \begin{pmatrix}
x_1 & x_2 & x_3 & \cdots & x_n\\
\end{pmatrix}
\times
\begin{pmatrix}
w_1\\
w_2\\
w_3\\
\vdots\\
w_n
\end{pmatrix} + b$$
```
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
hypothesis = x_train.mm(W) + b
```
```requires_grad=True``` indicates that the parameter will be trained.<br>
Last line is identical to ```hypothesis = x_train.matmul(W) + b```

#Optimizer and Learning Rate
We'll use SGD and Learning Rate of 1e-5.
```
optimizer = torch.optim.SGD([W, b], lr = 1e-5)
```

# Cost(MSE)

$$cost(W, b) = {1\over m}\sum \limits _{i=1} ^m (H(x^{(i)}) - y^{(i)})^2
$$

```
cost = torch.mean((hypothesis - y_train)**2)
```

# Update Weight and Bias

$\Delta w_j = {\partial cost \over {\partial w_j}}
= {2\over m}\sum \limits _{i=1} ^m (H(x^{(i)})x_j - y^{(i)})$<br>

$w_j : = w_j - lr \Delta w_j$

$\Delta b = {\partial cost \over {\partial b}}
= {2\over m}\sum \limits _{i=1} ^m (H(x^{(i)}) - y^{(i)})$<br>

$b : = b - lr \Delta b$

Initialize all the gradients to zero:
```
optimizer.zero_grad()
```

Backward Propagation:

```
cost.backward()
```

Update:
```
optimizer.step()
```


In [ ]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = torch.optim.SGD([W, b], lr = 1e-5)
print(f'Train data:\t\t\t{y_train.squeeze().detach()}')
nb_epoch = 1000
for epoch in range(nb_epoch):
  # Calculate H(x)
  hypothesis = x_train.mm(W) + b
  # Calculate cost
  cost = torch.mean((hypothesis - y_train)**2)
  # Update Weights and Bias
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  if epoch % 100 == 99:
    print(f'epoch: {epoch:4d}\t|hypothesis: {hypothesis.squeeze().detach()}\t|cost: {cost.item():.4f}')
print('train finished')

Train data:			tensor([152., 185., 180., 196., 142.])
epoch:   99	|hypothesis: tensor([152.7695, 183.6982, 180.9592, 197.0628, 140.1332])	|cost: 1.5643
epoch:  199	|hypothesis: tensor([152.7277, 183.7271, 180.9466, 197.0518, 140.1727])	|cost: 1.4982
epoch:  299	|hypothesis: tensor([152.6870, 183.7551, 180.9344, 197.0410, 140.2112])	|cost: 1.4356
epoch:  399	|hypothesis: tensor([152.6474, 183.7825, 180.9225, 197.0305, 140.2487])	|cost: 1.3763
epoch:  499	|hypothesis: tensor([152.6089, 183.8091, 180.9109, 197.0202, 140.2852])	|cost: 1.3200
epoch:  599	|hypothesis: tensor([152.5714, 183.8349, 180.8997, 197.0102, 140.3208])	|cost: 1.2667
epoch:  699	|hypothesis: tensor([152.5350, 183.8601, 180.8888, 197.0004, 140.3554])	|cost: 1.2162
epoch:  799	|hypothesis: tensor([152.4995, 183.8846, 180.8781, 196.9908, 140.3891])	|cost: 1.1683
epoch:  899	|hypothesis: tensor([152.4651, 183.9085, 180.8678, 196.9815, 140.4220])	|cost: 1.1229
epoch:  999	|hypothesis: tensor([152.4315, 183.9316, 180.8578, 19

In [ ]:
x_test = torch.FloatTensor([72, 81, 76]).unsqueeze(0)
print(f'W: {W}\n b: {b}')
print(f'y_test: {x_test.mm(W) + b}')

W: tensor([[0.7179],
        [0.6126],
        [0.6801]], requires_grad=True)
 b: tensor([0.0092], requires_grad=True)
y_test: tensor([[153.0060]], grad_fn=<AddBackward0>)


# nn.Module implementation

Instead of designing model from the scratch everytime, we could inherit the nn.Module class to build our model.<br>
Set the input and output for the linear layer.<br>
Write ```forward()```<br>
```backward()``` will automatically propagate from cost.

In [ ]:
class MultivariateLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(3, 1) ## Designate input and output dimensions.

    def forward(self, x):
        return self.linear(x)

# nn.functional implementation

We can use predefined cost function provided in nn.functional.<br>
Changing cost function becomes much easier this way.

Instead of explicitly writing MSE Loss,
```
cost = torch.mean((hypothesis - y_train)**2)
```
implement the mse_loss method provided in nn.functional.
```
cost = torch.nn.functional.mse_loss(hypothesis, y_train)
```

In [ ]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

model = MultivariateLinearRegressionModel()

optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
print(f'Train data:\t\t\t{y_train.squeeze().detach()}')
nb_epoch = 1000
for epoch in range(nb_epoch):

  # Calculate H(x)
  hypothesis = model(x_train)
  # Calculate cost
  cost = torch.nn.functional.mse_loss(hypothesis, y_train)
  
  # Initialize all the gradients to zero
  optimizer.zero_grad()
  # Backward Propagation
  cost.backward()
  # Update
  optimizer.step()
  if epoch % 100 == 99:
    print(f'epoch: {epoch:4d}\t|hypothesis: {hypothesis.squeeze().detach()}\t|cost: {cost.item():.4f}')
print('train finished')

Train data:			tensor([152., 185., 180., 196., 142.])
epoch:   99	|hypothesis: tensor([154.0974, 182.8673, 181.5178, 196.6237, 139.6704])	|cost: 3.4134
epoch:  199	|hypothesis: tensor([154.0283, 182.9146, 181.4966, 196.6084, 139.7325])	|cost: 3.2428
epoch:  299	|hypothesis: tensor([153.9610, 182.9607, 181.4760, 196.5935, 139.7930])	|cost: 3.0812
epoch:  399	|hypothesis: tensor([153.8956, 183.0055, 181.4559, 196.5790, 139.8519])	|cost: 2.9281
epoch:  499	|hypothesis: tensor([153.8319, 183.0492, 181.4364, 196.5649, 139.9092])	|cost: 2.7831
epoch:  599	|hypothesis: tensor([153.7700, 183.0917, 181.4174, 196.5511, 139.9649])	|cost: 2.6458
epoch:  699	|hypothesis: tensor([153.7096, 183.1330, 181.3989, 196.5378, 140.0192])	|cost: 2.5156
epoch:  799	|hypothesis: tensor([153.6509, 183.1733, 181.3810, 196.5247, 140.0720])	|cost: 2.3924
epoch:  899	|hypothesis: tensor([153.5938, 183.2124, 181.3634, 196.5121, 140.1234])	|cost: 2.2756
epoch:  999	|hypothesis: tensor([153.5382, 183.2506, 181.3464, 19

In [ ]:
x_test = torch.FloatTensor([72, 81, 76]).unsqueeze(0)
w, b = model.parameters()
print(f'w: {w},\nb:{b}')
print(f'y_test: {model(x_test).item()}')

w: Parameter containing:
tensor([[0.8476, 0.8337, 0.3385]], requires_grad=True),
b:Parameter containing:
tensor([-0.4140], requires_grad=True)
y_test: 153.8621826171875
